Essential Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import dill
from keras.preprocessing.image import ImageDataGenerator
import shutil
import os
import pandas as pd

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

DIR_DATASET = '/kaggle/input/tomato-short-dataset-5000'
TRAIN_DIR = DIR_DATASET + '/train'
TEST_DIR = DIR_DATASET + '/test'
VAL_DIR = DIR_DATASET + '/val'

In [ ]:
# Convolutional layer = Conv2D
# Pooling layer 
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 1000

base_model = VGG16(include_top=False,input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS))
model = models.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(n_classes,activation='softmax'))
model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
# checkpoints
checkpoint_path = "checkpoints2/cp_{epoch:02d}"
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
latest_epoch = 0
if latest:
    latest_epoch = int(latest[-2:])
    print("Latest Epoch", latest_epoch)
    model.load_weights(latest)
    print("Old Score", model.evaluate(test_ds))

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
df_train=pd.read_csv('/kaggle/input/custom-weights-tomato-5000/weights/train_weights.csv')
df_test=pd.read_csv('/kaggle/input/custom-weights-tomato-5000/weights/test_weights.csv')
df_val=pd.read_csv('/kaggle/input/custom-weights-tomato-5000/weights/val_weights.csv')

In [ ]:
# train_ds = train_datagen.flow_from_directory(TRAIN_DIR, 
#                                              target_size=(IMAGE_SIZE, IMAGE_SIZE), 
#                                              batch_size = BATCH_SIZE,
#                                              class_mode='sparse')
# test_ds = train_datagen.flow_from_directory(TEST_DIR, 
#                                              target_size=(IMAGE_SIZE, IMAGE_SIZE), 
#                                              batch_size = BATCH_SIZE,
#                                              class_mode='sparse')
# val_ds = train_datagen.flow_from_directory(VAL_DIR, 
#                                              target_size=(IMAGE_SIZE, IMAGE_SIZE), 
#                                              batch_size = BATCH_SIZE,
#                                              class_mode='sparse')

In [ ]:
train_ds = train_datagen.flow_from_dataframe(dataframe=df_train, 
                                         directory=TRAIN_DIR, 
                                         x_col='full_path', 
                                         y_col='class',
#                                          weight_col='norm_weight',
                                         class_mode='sparse')

test_ds = test_datagen.flow_from_dataframe(dataframe=df_test, 
                                         directory=TEST_DIR, 
                                         x_col='full_path', 
                                         y_col='class',
#                                          weight_col='norm_weight',
                                         class_mode='sparse')

val_ds = val_datagen.flow_from_dataframe(dataframe=df_val, 
                                         directory=VAL_DIR, 
                                         x_col='full_path', 
                                         y_col='class',
#                                          weight_col='norm_weight',
                                         class_mode='sparse')



In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)

In [ ]:
score = model.evaluate(test_ds)

In [ ]:
history.params
# history.history['accuracy']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
plt.figure(figsize=(8,5))
plt.subplot(1,2,1)
plt.plot(range(0, len(acc)),acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(range(0,len(loss)),loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

In [ ]:
# # Make Predictions
# '''
# for images_batch, labels_batch in test_ds.take(1):
#     first_image = images_batch[0].numpy().astype('uint8')
#     first_label = labels_batch[0]
    
#     print("First image to predict")
#     plt.imshow(first_image)
#     print("actual label:", class_names[first_label])
    
#     batch_prediction = model.predict(images_batch)
#     predicted_class_index = np.argmax(batch_prediction[0])
#     print("Predicted label: ", class_names[predicted_class_index])
# '''

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)  #creating a batch
    
    predictions = model.predict(img_array)
    
    predicted_class =  class_names[np.argmax(predictions[0])]
    confidence = round(100*(np.max(predictions[0])),2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(14,14))
for images in test_ds:
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.axis('off')
        
        predicted_class, confidence = predict(model, images[i].numpy())
        
        actual_class = class_names[labels[i]]
        
        plt.title(f"Actual: {actual_class},\nPredicted: {predicted_class}.\nConfidence: {confidence}%")
        

In [ ]:
# save model
model_version=2
model.save(f"/kaggle/working/models/{model_version}",save_format='h5')
# model.save("../models/model1")

In [ ]:
from sklearn.metrics import confusion_matrix

#Predict
y_prediction = model.predict(test_ds)

#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(test_ds.labels, y_prediction , normalize='pred')

In [ ]:
print(history.history)